The previous steps in the data organization created a dictionary called OVERAL_DICT that had the SERIES ID as the key and the nodule/non-nodule co-ordinates as the values. Unfortunately, the actual dicom images were not named by the SERIES ID and due to the complicated folder structure that the whole dataset comes in, they had to be tied to the SERIES ID found in the dictionary. 

Here, a set of lists were created that : 
1. Contained the actual dicom image path (REL_IMGS) - 13 of them 
2. A corresponding list containing the annotations (CORDS) - 13 of them 

In [ ]:
import xml.etree.ElementTree as ET
import dicom
import matplotlib.pyplot as plt
import glob
import pickle
import numpy as np
import sys, os
import pylab
import time 

In [ ]:
ALL_DCM = []
lidc_folders = []
for patient in range(1,1013):
    if patient>0 and patient<10: 
        mainf = 'LIDC-IDRI-000'+str(patient)
    if patient<100 and patient >=10: 
        mainf = 'LIDC-IDRI-00'+str(patient)
    if patient>=100 and patient<1000:
        mainf = 'LIDC-IDRI-0'+str(patient)
    if patient>=1000 and patient<1013:
        mainf = 'LIDC-IDRI-'+str(patient)
    lidc_folders.append(mainf)
        
    lod = [x[0] for x in os.walk("DOI/"+str(mainf))]
    for d in lod: 
        if len(glob.glob(str(d)+"/*.dcm"))!=0:
            ALL_DCM.extend(glob.glob(str(d)+"/*.dcm"))
len(ALL_DCM)
with open('ALL_DCM_IMAGES.pkl','wb') as adm:
    pickle.dump(ALL_DCM, adm)

In [ ]:
OVERALL_DICT = pickle.load( open( "OVERALL_DICT.pkl", "rb" ) )
SERIES = pickle.load( open( "SERIES.pkl", "rb" ))
ALL_DCM = pickle.load( open( "ALL_DCM_IMAGES.pkl", "rb" ))

In [ ]:
REL_IMGS = []
CORDS = []
tic = time.time()
i = 0 
for n in range(0,13):
    for series in SERIES[n*100:(n*100) + 100]:

        print(i)
        for image in ALL_DCM:
            if series in str(image):
                if dicom.read_file(image)[0x08,0x18].value in OVERALL_DICT[series].keys(): 
                    CORDS.append(OVERALL_DICT[series][dicom.read_file(image)[0x08,0x18].value])
                    REL_IMGS.append(image)

        i+=1  



    toc = time.time()
    print(toc - tic)

    with open('REL_IMGS'+str(n)+'.pkl','wb') as ri:
        pickle.dump(REL_IMGS, ri)
    with open('CORDS'++str(n)+'.pkl','wb') as co:
        pickle.dump(CORDS, co)

In [ ]:
#Total number of images arranged 
im = 0 
for r in range(0,13):
    fn = 'REL_IMGS'+str(r)+'.pkl'
    RI = pickle.load(open(fn,'rb'))
    print('Total relevant images: ',len(RI))
    im +=len(RI)
im 